<a href="https://colab.research.google.com/github/kconstable/market_predictions/blob/main/predict_market_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict Market Prices
This notebook was created to develop funcitons for the following
+ Create an ensemble model to predict prices (LSTM, Facebook Prophet, Naiive Models)
+ Create visuals to display actual vs predicted values for the ensemble
+ Run a historical back-test of predicted values
+ to support a plotly/dash app that automatally gets new prices daily, predicts the next 3 days of prices, and updates the actual/predicted history file
+ Some functions in this notebook are repeated from the market_data notebook

## Import Libraries

In [ ]:
# install the prophet library
import sys
!{sys.executable} -m pip install fbprophet
from fbprophet import Prophet

In [ ]:
import pandas as pd
import numpy as np
import requests
import time
import pickle
import re
import datetime
from datetime import timedelta
from tabulate import tabulate
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import adfuller


# alphavalue key
with open('/content/drive/MyDrive/Colab Notebooks/data/av_key.txt') as f:
    key = f.read().strip()

## Functions

### Data Collection
+ These functions are repeated from the market_data and lstm files

In [ ]:
def get_economic_indicators(funct,key,interval=None,maturity=None,throttle=0):
  """
  Returns Economic Indicator Data with missing values interpolated between dates
  Monthly Data:
    NONFARM_PAYROLL, INFLATION_EXPECTATION,CONSUMER_SENTIMENT,UNEMPLOYMENT
  Daily, Weekly, Monthly Data:  
    FEDERAL_FUNDS_RATE = interval (daily,weekly,monthly)
    TREASURY_YIELD = interval (daily, weekly, monthly), 
                     maturity (3month, 5year, 10year, and 30year)
  """
  
  # query strings
  # Monthly Data:
  if funct in ['NONFARM_PAYROLL','INFLATION_EXPECTATION','CONSUMER_SENTIMENT','UNEMPLOYMENT']:
    url = f'https://www.alphavantage.co/query?function={funct}&apikey={key}'

  # Daily, Weekly or Monthly Data:
  # Interest Rates
  if funct == 'FEDERAL_FUNDS_RATE':
    url = f'https://www.alphavantage.co/query?function={funct}&interval={interval}&apikey={key}'

  # Treasury Yield  
  if funct == 'TREASURY_YIELD':
    url = f'https://www.alphavantage.co/query?function={funct}&interval={interval}&maturity={maturity}&apikey={key}'

  # pull data
  r = requests.get(url)
  time.sleep(throttle)
  d = r.json()

  # convert to df
  df = pd.DataFrame(d['data'])

  # move date to a datetime index
  df.date = pd.to_datetime(df.date)
  df.set_index('date',inplace=True)

  # add the ticker name and frequency
  df['name'] = d['name']
  df['interval']=d['interval'] 

  # clean data & interpolate missing values
  # missing data encoded with '.'
  # change datatype to float
  df.replace('.',np.nan,inplace=True)
  df.value = df.value.astype('float')

  # missing data stats
  missing =sum(df.value.isna())
  total =df.shape[0]
  missing_pct = round(missing/total*100,2)

  # interpolate using the time index
  if missing >0:
    df.value.interpolate(method='time',inplace=True)
    action = 'interpolate'
  else:
    action = 'none'

  # Print the results
  if maturity is not None:
    summary = ['Economic Indicator',funct+':'+maturity,str(total),str(missing),str(missing_pct)+'%',action]
  else:
    summary = ['Economic Indicator',funct,str(total),str(missing),str(missing_pct)+'%',action]


  return {'summary':summary,'data':df}

In [ ]:
def get_technical_indicators(symbol,funct,key,interval,time_period=None,throttle=0):
  """
  Returns Technical Indicators (only works for stocks, not cyrpto)
  MACD:   symbol,interval
  RSI:    symbol,interval,time_period
  BBANDS: symbol,interval,time_period

  Parameters:
          interval: (1min, 5min, 15min, 30min, 60min, daily, weekly, monthly)
          series_type: (open, close,high,low)-default to close
          timer_periods: Integer
  """
  # build the query string
  if funct =='MACD':
    url = f'https://www.alphavantage.co/query?function={funct}&symbol={symbol}&interval={interval}&series_type=close&apikey={key}'
  if funct in ['RSI','BBANDS']:
    url = f'https://www.alphavantage.co/query?function={funct}&symbol={symbol}&interval={interval}&series_type=close&time_period={time_period}&apikey={key}'

  # request data as json, convert to dict, pause request to avoid the data throttle
  r = requests.get(url)
  time.sleep(throttle)
  d = r.json()

  # extract to a df, add the indicator name, convert the index to datetime
  df = pd.DataFrame(d[f'Technical Analysis: {funct}']).T
  df.index = pd.to_datetime(df.index)

  # convert the data to float
  for col in df.columns:
    df[col] = df[col].astype('float')

  # check for missing data
  missing = df.isnull().any().sum()
  total = len(df)
  missing_pct = round(missing/total*100,2)


  # Print the results
  summary=['Technical Indicator',funct,str(total),str(missing),str(missing_pct)+'%','none']

  return {'summary':summary,'data':df}

In [ ]:
def get_crypto_data(symbol,key):
  """
  Pulls daily crypto prices from alpha advantage.
  Inputs:
    symbol: ETH, BTC, DOGE
    key:    The alpha advantage API key
  Output:
    a dataframe of crypto prices: open,high,low, close, volume
  """
  # build query string, get data as json and convert to a dict
  url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market=CAD&apikey={key}'
  r = requests.get(url)
  d = r.json()

  # extract data to df
  df=pd.DataFrame(d['Time Series (Digital Currency Daily)']).T

  # remove columns not required
  # returns the price in two currencies, just keep USD
  cols = [c for c in df.columns if '(CAD)' not in c]
  df=df.loc[:, cols]
  df.columns = ['open','high','low','close','volume','marketcap']
  df.drop(['marketcap'],axis=1,inplace=True)

  # change data types
  df.index = pd.to_datetime(df.index)

  # convert datatype to float
  for col in ['open','high','low','close','volume']:
    df[col] = df[col].astype('float')

  # add the cyrpto name
  df['symbol'] = d['Meta Data']['3. Digital Currency Name']

  return df

In [ ]:
def calc_bollinger(df,feature,window=20,st=2):
  """
  Calculates bollinger bands for a price time-series.  Used for crypto currencies
  Input: 
    df     : A dataframe of time-series prices
    feature: The name of the feature in the df to calculate the bands for
    window : The size of the rolling window.  Defaults to 20 days with is standard
    st     : The number of standard deviations to use in the calculation. 2 is standard 
  Output: 
    Returns the df with the bollinger band columns added
  """

  # rolling mean and stdev
  rolling_m  = df[feature].rolling(window).mean()
  rolling_st = df[feature].rolling(window).std()

  # add the upper/lower and middle bollinger bands
  df['b-upper']  = rolling_m + (rolling_st * st)
  df['b-middle'] = rolling_m 
  df['b-lower']  = rolling_m - (rolling_st * st)

In [ ]:
def calc_rsi(df,feature='close',window=14):
  """
  Calculates the RSI for the input feature
  Input:
    df      : A dataframe with a time-series of prices
    feature : The name of the feature in the df to calculate the bands for
    window  : The size of the rolling window.  Defaults to 14 days which is standard
  Output: 
    Returns the df with the rsi band column added
  """
  # RSI
  # calc the diff in daily prices, exclude nan
  diff =df[feature].diff()
  diff.dropna(how='any',inplace=True)

  # separate positive and negitive changes
  pos_m, neg_m = diff.copy(),diff.copy()
  pos_m[pos_m<0]=0
  neg_m[neg_m>0]=0

  # positive/negative rolling means
  prm = pos_m.rolling(window).mean()
  nrm = neg_m.abs().rolling(window).mean()

  # calc the rsi and add to the df
  ratio = prm /nrm
  rsi = 100.0 - (100.0 / (1.0 + ratio))
  df['rsi']=rsi

In [ ]:
def calc_macd(df,feature='close'):
  """
  Calculates the MACD and signial for the input feature
  Input:
    df      : A dataframe with a time-series of prices
    feature : The name of the feature in the df to calculate the bands for
  Output: 
    Returns the df with the macd columns added
  """
  ema12 = df[feature].ewm(span=12,adjust=False).mean()
  ema26 = df[feature].ewm(span=26,adjust=False).mean()
  df['macd']=ema12-ema26
  df['macd_signal'] = df['macd'].ewm(span=9,adjust=False).mean()

In [ ]:
def get_ticker_data(symbol,key,outputsize='compact',throttle=0):
  """
  Returns daily data for a stock (symbol)
    outputsize: compact(last 100) or full (20 years)
    key: apikey
    symbols: OILK (oil ETF),BAR(gold ETF),VXZ (volatility ETF)
  """
  url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize={outputsize}&apikey={key}'
  r = requests.get(url)
  time.sleep(throttle)
  d = r.json()

  # extract data to a df
  df = pd.DataFrame(d['Time Series (Daily)']).T
  df.columns = ['open','high','low','close','volume']
  df['symbol'] = d['Meta Data']['2. Symbol']

  # change data types
  df.index = pd.to_datetime(df.index)

  # convert datatype to float
  for col in ['open','high','low','close','volume']:
    df[col] = df[col].astype('float')

  # Calculate missing data
  missing = sum(df.close.isna())
  total = df.shape[0]
  missing_pct = round(missing/total*100,2)

  # Print the results
  summary = ['Ticker',symbol,str(total),str(missing),str(missing_pct)+'%','none']

  return {'summary':summary,'data':df}

In [ ]:
def get_consolidated_stock_data(symbol,key,config,outputsize='compact',throttle=30,dropna=True):
  """
  Pulls data from alpha advantage and consolidates
  API Limitations: 5 API requests per minute and 500 requests per day
  Inputs:
    symbol: stock ticker
    key   : api key
    config: dictionary which lists the economic, technical and commodities to pull
    outputsize: compact(latest 100) or full (up to 20 years of daily data)
    throttle: number of seconds to wait between api requests
    dropna: True/False, drops any records with nan
  Output:
    A dataframe with consolidated price data for the symbol + economic/technical
    indicators and commodity prices
  """

  # Result header and accumulator
  header = ['Type','Data','Total','Missing',' % ','Action']
  summary =[]

  # Get stock prices
  try:
    results  = get_ticker_data(symbol,key,outputsize,0)
    dff = results['data']
    summary.append(results['summary'])
    print(f'Complete:===>Ticker:{symbol}')
  except:
    print(f'Error:===>Ticker:{symbol}')



  # Get Commodity prices
  # ****************************************************************************
  for commodity in config['Commodities']:
    try:
      # get prices
      results = get_ticker_data(commodity,key,outputsize,throttle)
      df = results['data']
      summary.append(results['summary'])
      print(f'Complete:===>Commodity:{commodity}')


      # rename close to commodity name, remove unneeded columns and join with 
      # the stock prices by date
      df.rename(columns={'close':commodity},inplace=True)
      df.drop(['open','high','low','volume','symbol'],axis=1,inplace=True)
      dff = dff.join(df,how='left')
    except:
      print(f"Error===>Commodity:{commodity}")


  # Economic Indicators
  # ****************************************************************************
  # loop through the config to pull the requested data
  for indicator,values in config['Economic'].items():
    if indicator == 'TREASURY_YIELD':
      for tr in values:
        try:
          results = get_economic_indicators(indicator,key,interval=tr['interval'],maturity=tr['maturity'],throttle=throttle)
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}:{tr['maturity']}")

          df = results['data']
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": tr['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}:{tr['maturity']}")
   
    else: 
      # daily
      if values['interval']=='daily':
        try:
          results = get_economic_indicators(indicator,key,interval=values['interval'],throttle=throttle)
          df = results['data']
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}")

          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")
  
      else: 
        try:
          # monthly or weekly
          results = get_economic_indicators(indicator,key,throttle=throttle)
          summary.append(results['summary'])
          df = results['data']
          print(f"Complete:===>{indicator}")

          # reindex to daily, fill missing values forward
          days = pd.date_range(start = min(df.index),end =max(df.index),freq='D')
          df =df.reindex(days,method = 'ffill')
      
          # join with the other data
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")

  # # Technical Indicators
  # ****************************************************************************
  for indicator,values in config['Technical'].items():
    try:
      results = get_technical_indicators(symbol,indicator,key,values['interval'],values['time_period'],throttle)
      df = results['data']
      summary.append(results['summary'])

      dff = dff.join(df,how='left')
      print(f"Complete:===>{indicator}")
    except:
      print(f"Error===>{indicator}")

  
  # clean column names
  dff.rename(columns={"Real Upper Band":'b-upper',
                      "Real Lower Band":'b-lower',
                      "Real Middle Band":"b-middle",
                      "RSI":"rsi",
                      "MACD_Hist":"macd_hist",
                      "MACD_Signal":"macd_signal",
                      "MACD":"macd"
                      },inplace=True)
      

  # Fill in any missing data after joining all datasets
  dff.fillna(method='bfill',inplace=True,axis = 0)

  # drop rows with missing commodity prices
  if dropna:
    dff.dropna(how='any',inplace=True)

  # print the results table
  print("\n\n")
  print(tabulate(summary,header))

  return dff

In [ ]:
def get_consolidated_crypto_data(symbol,key,config,boll_window=20,boll_std=2,rsi_window=14,throttle=30,dropna=True):
  """
  Pulls data from alpha advantage and consolidates
  API Limitations: 5 API requests per minute and 500 requests per day
  Inputs:
    symbol: crypto ticker
    key   : api key
    config: dictionary which lists the economic indicators and commodities to pull
    throttle: number of seconds to wait between api requests
    dropna: True/False, drops any records with nan
  Output:
    A dataframe with consolidated price data for the symbol + economic/technical
    indicators and commodity prices
  """

  # Result header and accumulator
  header = ['Type','Data','Total','Missing',' % ','Action']
  summary =[]

  # Get crypto prices
  try:
    dff  = get_crypto_data(symbol,key)
    
    # add month feature
    dff['month'] = dff.index.month

    print(f'Complete:===>Crypto:{symbol}')

  except:
    print(f'Error:===>Crypto:{symbol}')


  # Get Commodity prices
  # ****************************************************************************
  for commodity in config['Commodities']:
    try:
      # get prices
      results = get_ticker_data(commodity,key,'full',throttle)
      df = results['data']
      summary.append(results['summary'])
      print(f'Complete:===>Commodity:{commodity}')


      # rename close to commodity name, remove unneeded columns and join with 
      # the stock prices by date
      df.rename(columns={'close':commodity},inplace=True)
      df.drop(['open','high','low','volume','symbol'],axis=1,inplace=True)
      dff = dff.join(df,how='left')
    except:
      print(f"Error===>Commodity:{commodity}")


  # Economic Indicators
  # ****************************************************************************
  # loop through the config to pull the requested data
  for indicator,values in config['Economic'].items():
    if indicator == 'TREASURY_YIELD':
      for tr in values:
        try:
          results = get_economic_indicators(indicator,key,interval=tr['interval'],maturity=tr['maturity'],throttle=throttle)
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}:{tr['maturity']}")

          df = results['data']
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": tr['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}:{tr['maturity']}")
   
    else: 
      # daily
      if values['interval']=='daily':
        try:
      
          results = get_economic_indicators(indicator,key,interval=values['interval'],throttle=throttle)
          df = results['data']
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}")

          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")
  
      else: 
        try:
          # monthly or weekly
         
          results = get_economic_indicators(indicator,key,throttle=throttle)
          summary.append(results['summary'])
          df = results['data']
          print(f"Complete:===>{indicator}")

          # reindex to daily, fill missing values forward
          days = pd.date_range(start = min(df.index),end =max(df.index),freq='D')
          df =df.reindex(days,method = 'ffill')
      
          # join with the other data
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")

  # # Technical Indicators
  # ****************************************************************************
  calc_rsi(dff,'close',rsi_window)
  calc_bollinger(dff,'close',boll_window,boll_std)
  calc_macd(dff,'close')
      

  # Fill in any missing data after joining all datasets
  dff.fillna(method='bfill',inplace=True,axis = 0)

  # drop rows with missing commodity prices
  if dropna:
    dff.dropna(how='any',inplace=True)

  # print the results table
  print("\n\n")
  print(tabulate(summary,header))

  return dff

In [ ]:
def transform_stationary(df,features_to_transform,transform='log',verbose=False):
  """
  Transform time-series data using a log or boxcox transform.  Calculate the augmented
  dickey-fuller (ADF) test for stationarity after the transform
  Inputs:
    df: a dataframe of features
    features_to_transform: A list of features to apply the transform
    transform: The transform to apply (log, boxbox)
  Output
    Applies the transforms inplace in df
  """
  # transform each column in the features_to_transform list
  for feature in df.columns:
    if feature in features_to_transform:
      # log transform
      if transform=='log':
        df[feature] = df[feature].apply(np.log)
  
      # boxcox transform  
      elif transform=='boxcox':
        bc,_ = stats.boxcox(df[feature])
        df[feature] = bc
 
      else:
        print("Transformation not recognized")

  if verbose:      
    # check the closing price for stationarity using the augmented dicky fuller test
    t_stat, p_value, _, _, critical_values, _  = adfuller(df.close.values, autolag='AIC')
    print('\n\nAugmented Dicky Fuller Test for Stationarity')
    print("="*60)
    print(f'ADF Statistic: {t_stat:.2f}')
    for key, value in critical_values.items():
      print('Critial Values:')
      if t_stat < value:
        print(f'   {key}, {value:.2f} => non-stationary')
      else:
        print(f'   {key}, {value:.2f} => stationary')



In [ ]:
def shift_features(df,features_shift):
  """
  Shifts features by time periods to convert them to lagged indicators
  Input:
    df: dataframe of features
    features_shift: dictionary  of {feature:period shift}
  Output:
    df: original dataframe + the shifted features
  """
  dff = df.copy()
  for feature,shift in features_shift.items():
    t_shift = pd.DataFrame(dff[feature].shift(periods=shift))
    dff =dff.join(t_shift,how='left',rsuffix='_shift')

  # remove nan introducted with lag features
  dff.dropna(how='any',inplace=True)

  return dff

In [ ]:
def prepare_data(df,n_steps,features=[],verbose=False):
  """
  Filter, scale and convert dataframe data to numpy arrays

  Inputs: 
    df       => A dataframe of observations with features and y-labels
    y        => The name of the column that is the truth labels
    features => A list of features.  Used to subset columns

  Outputs:
    scaled_y => numpy array of the y-label data
    scaled_x => numpy array of the training features

  """

  # subset the latest n_steps rows to be used for prediction
  df = df.iloc[0:n_steps,:]

  # reverse the index such that dates are in chronological order
  df = df.iloc[::-1]

  # Subset features, get the y-label values
  df_y = df['close']
  df_X = df[features]

  # replace the date index with an integer index
  idx_dates = df.index
  df_X.reset_index(drop=True,inplace=True)

  # convert to numpay arrays
  array_X = np.array(df_X)
  array_y = np.array(df_y).reshape(-1,1)

  if verbose:
    # print the output
    print("\nData Preparation")
    print("="*60)
    print(f"=> {len(features)} Features")
    print(f"=> Input Dimensions :{array_X.shape}")
    print("\n")

  return idx_dates, array_y,array_X

# Ensemble Methods
+ The LSTM model uses previous technical indicators and clearly shows a lagged trend following behaviour
+ this is a common issue with LSTM models as it treats data from 40 days ago equally as data from the previous day
+ To compensate the following measures were taken:
  + shorten technial indicator windows for bollinger bands (20 lag to 10 day lag) and RSI (14 day lag to 7 day lag)
  + reduce the date input window from previous 40 days to the previous 25 days
  + reduce the prediction window from 5 days to 3 days
  + introduce an ensemble method that is better at following recent trends
  + introduce a factor to weight the previous days close into the next three days of prediction



## FB Prophat
+ facebooks prophet algorithm was used to capture trend and seasonality in the price movements.
+ prophet is a variation of an ARIMA model which only uses the time-series of closing prices to estimate the next closing price

In [ ]:
def get_prophet_df(df):
  """
  Convert a dataframe into prophet format
  Input:
    Dataframe of prices
  Output:
    dataframe matching the prohet requirements of datestamp,
    target variable(ds,y)
  """

  df_prophet = df[['close']].copy()
  df_prophet.reset_index(inplace= True)
  df_prophet.rename(columns={'index':'ds','close':'y'},inplace=True)

  return df_prophet


def create_prophet_model(df):
  """
  Creates and fits a FB Prophet model on the input time-series
  Input:
    df: a dataframe containing the closing price time-series
  Output:
    a trained prophet model
  """
  # convert the data into prophet format
  df_prophet = get_prophet_df(df,growth = 'linear')

  # init the prophet model and fit to the dataset
  m = Prophet(daily_seasonality=False)
  m.fit(df_prophet)

  return m



def get_prophet_forecast(df,periods=3,visualize= False):
  """
  Predicts the closing price using FB Prophat algorithm for the input 
  number of periods
  Input:
    df: a dataframe of closing prices
    periods: the number of periods to predict
    visualize: boolen, determines if a plot should be displayed
  Output:
    a dataframe of predcited closing prices
  """

  # get lastest date
  latest_date =df.index.max()

  # convert the data into prophet format
  df_prophet = get_prophet_df(df)

  # init the prophet model and fit to the dataset
  model = Prophet(daily_seasonality=False)
  model.fit(df_prophet)

  # create a df to hold the predictions
  future = model.make_future_dataframe(periods=periods)


  # forecast the data
  forecast = model.predict(future[future['ds']>latest_date])

  # show history + forcast
  if visualize:
    model.plot(forecast)

  # subset rows/cols
  # df_subset = forecast[forecast['ds']>latest_date]
  df_subset = forecast.loc[:,('ds','yhat','yhat_lower','yhat_upper')]

  return df_subset

In [ ]:
def update_price(df_hist,date,value,type='actual'):
  """
  Updates the price in the history dataframe
  Input: 
    df_hist: the dataframe that contains the history of prices/predicitons/metrics
    date: the date to update
    value: the price to update
    type: actual or predicted price to update
  """
  # update the price as of the date
  # should be yesterdays price
  df_hist.at[date,type]=value


In [ ]:
def make_ensemble_predictions(df,lstm_model,scaler,scaled_X,n_steps,n_features,n_pred,start_date,weights):
  """
  Predict the next n_pred days with n_steps of daily data as input to the model
  Input:
    lstm_model: A trained LSTM model
    scaler: The scaler used
    scaled_X: scaled input features
    n_steps: the number of input days used in the model
    n_features: the number of features used in the model
    n_pred: the number of days predicted in the model
    start_date: the start date of the prediction window
  Output:
    a data frame of predicted prices
  """
  # LSTM Prediction
  # Predict the prices
  y_pred_scaled = lstm_model.predict(scaled_X.reshape(1,n_steps,n_features))

  # convert units back to the original scale
  y_pred_unscaled = scaler.inverse_transform(y_pred_scaled)

  # convert from log transform back to original scale
  y_pred_np = np.exp(y_pred_unscaled)

  # set the date index
  pred_dates = pd.date_range(start_date + datetime.timedelta(days=1), periods=n_pred,freq='D').tolist()


  # convert to dataframe
  lstm = pd.DataFrame(y_pred_np.T,columns=['lstm'])
  lstm['actual']= np.nan
  lstm['date'] = pred_dates
  lstm.set_index(['date'],inplace=True)
  lstm.index = pd.to_datetime(lstm.index)
  lstm =lstm[['actual','lstm']]


  # FB Prophat Prediction
  fbp = get_prophet_forecast(df,n_pred,False)
  fbp['yhat'] = np.exp(fbp['yhat'])

  # Get last closing price
  last_close = np.exp(df.loc[df.index.max()]['close'])
  last_close_lst = [last_close for p in range(n_pred)]


  # create ensemble prediction
  df_ensemble = pd.DataFrame(list(zip(lstm.lstm,fbp.yhat,last_close_lst)),columns=['lstm','fbp','last_close'])
  df_ensemble['date'] = lstm.index
  df_ensemble.set_index('date',inplace=True)
  df_ensemble['ens'] = df_ensemble['last_close']*weights['last_close']+df_ensemble['lstm']*weights['lstm']+df_ensemble['fbp']*weights['fbp']
  # df_ensemble['ens'] = df_ensemble['lstm']*weights['lstm'] + df_ensemble['fbp']*weights['fbp'] + df_new['last_close']*weights['last_close']


  return df_ensemble[['lstm','fbp','last_close','ens']]

In [ ]:
def roll_ensemble_predictions(df_new,df_pred=None,df_hist =None):
  """
  Updates previous predicted prices with actual prices, and adds
  the next n_pred prediction window
  Input:
    df_new: The new dataset of inputs
    df_pred: A dataframe of predicted prices
    df_hist: A dataframe that stores the actual/predicted prices 
            (the output of this function)
  Output:
    A dataframe of up-to-date prices with the next prediction window.
    Includes the daily and cumulative prediction error
  """
  # First time creating history file
  if df_pred is None and df_hist is None:
    # create the initial history of prices (without predictions)
    df_hist_new = pd.DataFrame(df_new['close'],columns=['close'])
    df_hist_new.columns = ['actual']
    df_hist_new['last_close'] = np.nan
    df_hist_new['last_close_diff']= np.nan
    df_hist_new['last_close_cum']=np.nan
    df_hist_new['lstm']=np.nan
    df_hist_new['lstm_diff']=np.nan
    df_hist_new['lstm_cum']=np.nan
    df_hist_new['fbp']=np.nan
    df_hist_new['fbp_diff']=np.nan
    df_hist_new['fbp_cum']=np.nan
    df_hist_new['ens']=np.nan
    df_hist_new['ens_diff']=np.nan
    df_hist_new['ens_cum']=np.nan


  else:
    # append to existing history file
    # make a copy of df_hist
    df = df_hist.copy()

    # Get yesterdays closing price
    yesterday = df_new.index.max()
    yesterdays_close = df_new.loc[yesterday,'close'].item()

    # update df_hist with yesterdays_close,
    update_price(df,yesterday,yesterdays_close,'actual')


    # remove old predictions 
    # yesterdays nan should have been replaced in the prevous step
    df = df[~df['actual'].isnull()]

    # add new predictions
    df_hist_new =pd.concat([df,df_pred])

    # calculate the difference between actual/predicted values
    # for current period and cumulative
    df_hist_new['last_close_diff'] = df_hist_new['last_close']-df_hist_new['actual']
    df_hist_new['last_close_cum'] = df_hist_new['last_close_diff'].cumsum()

    df_hist_new['lstm_diff'] = df_hist_new['lstm']-df_hist_new['actual']
    df_hist_new['lstm_cum'] = df_hist_new['lstm_diff'].cumsum()

    df_hist_new['fbp_diff'] = df_hist_new['fbp']-df_hist_new['actual']
    df_hist_new['fbp_cum'] = df_hist_new['fbp_diff'].cumsum()

    df_hist_new['ens_diff'] = df_hist_new['ens']-df_hist_new['actual']
    df_hist_new['ens_cum'] = df_hist_new['ens_diff'].cumsum()

    #sort by date
    df_hist_new.sort_index(inplace=True)

  return df_hist_new

In [ ]:
def plot_actual_ensemble(name,df,model='ens'):
  """
  Plots the prices as a time-series showing actual/predicted values with daily and 
  cumulative prediction errors
  Input:
    name: the name of the stock/crypto
    df: the historical dataframe (output from roll_ensemble_predictions)
    model: model to plot daily errors for (ens,fbp,lstm,last_price)
  """
  fig = make_subplots(rows=3, 
                      cols=1,
                      shared_xaxes=True,
                      vertical_spacing=0.1,
                      subplot_titles = ('Actual vs Predicted Closing Price','Daily Error','Cumulative Error'))
  # Actual prices
  fig.add_trace(go.Scatter(
      x=df.index,
      y=df.actual,
      fill='tozeroy',
      mode = 'lines',
      line =dict(color="#ccc"),
      name = 'Actual'),
      row=1,col=1
  )
  # predicted prices
  # ensemble
  fig.add_trace(go.Scatter(
      x=df.index,
      y=df.ens,
      mode = 'lines',
      line_color = 'orange',
      name= 'Ensemble'),
      row=1,col=1
  )
  # lstm
  fig.add_trace(go.Scatter(
      x=df.index,
      y=df.lstm,
      mode = 'lines',
      line_color = '#7cbf5a',
      name= 'LSTM'),
      row=1,col=1
  )
  # FB prophat
  fig.add_trace(go.Scatter(
      x=df.index,
      y=df.fbp,
      mode = 'lines',
      line_color = 'skyblue',
      name= 'FB Prophet'),
      row=1,col=1
  )


  # daily error
  # bar chart
  colors = {'ens':'orange','fbp':'skyblue','lstm':'#7cbf5a','last_close':'crimson'}
  fig.add_trace(go.Bar(
      x = df.index,
      y = df[f'{model}_diff'],
      name = f'{model} Error',
      marker_color = colors[model]
  ),row=2,col=1)

  # cumulative errors
  # ensemble
  fig.add_trace(go.Scatter(
      x=df.index,
      y=df[f'{model}_cum'],
      fill = 'tozeroy',
      line_color = 'orange',
      mode = 'lines',
      name = 'Cumulative Error- Ensemble Model'
  ),row=3,col=1)

  # LSTM
  fig.add_trace(go.Scatter( 
      x=df.index,
      y=df['lstm_cum'],
      fill = 'tozeroy',
      line_color = '#7cbf5a',
      mode = 'lines',
      name = 'Cumulative Error-LSTM Model'
  ),row=3,col=1)
  
  # FB p
  fig.add_trace(go.Scatter(
      x=df.index,
      y=df['fbp_cum'],
      fill = 'tozeroy',
      line_color = 'skyblue',
      mode = 'lines',
      name = 'Cumulative Error- FB Prophet Model'
  ),row=3,col=1)


  fig.update_layout(height=600, 
                    width=800, 
                    template = 'plotly_white',
                    title_text=f"{name}: Actual Vs. Predicted Prices")
  fig.show()


# Historical Back Test
The model was tested by creating a historical back test of predcitions from the beginning of January 2021 to September 2021
+ Pull all data to current date and create the df_hist file of actual prices
+ Start on Jan 4,2021 and make the 3 days of predictions
+ On Jan 5
  + get the actual price for Jan 5
  + calculate the prediction error
  + roll the model forward 1 day and make the next 3 day prediction
+ Repeat for everyday up until the end of september

### Bitcoin

In [ ]:
# Bitcoin
stock = 'BTC'
features_to_transform = ['open','high','low','close','b-upper','b-lower','b-middle','SPY','BOIL'] 
transform = 'log'
n_steps = 25
n_predict = 3

config ={'Economic':
         {'TREASURY_YIELD':[
                            {'interval':'daily','maturity':'10year','name':'yield10y'},
                            {'interval':'daily','maturity':'30year','name':'yield30y'}
                            ],
          },
         'Commodities':['SPY','BOIL']
         }

# Get the features used in the final model
df_features = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_features.pickle')
features = [f for f in df_features.columns if f not in ['symbol']]

# get full data history
# df_btc = get_consolidated_crypto_data(stock,key,config,boll_window=10,rsi_window=7)
df_new = df_btc[features].copy()


# get the trained lstm model
model = keras.models.load_model(f'/content/drive/MyDrive/Colab Notebooks/models/model_{stock}_final')


# initial hist file
start_date = '2021-01-04'
end_date = '2021-09-28' 
weights = {'lstm':0.448420,'fbp':0.446634,'last_close':0.104946}
df_tmp = df_new.loc[df_new.index <=start_date]
df_hist_new =roll_ensemble_predictions(df_tmp)
df_hist_new.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_ensemble_hist.pickle')

# backtest for sep 2021
# datelist = df_new.loc[start_date:end_date].index
datelist = df_new.loc[end_date:start_date].index
datelist = datelist[::-1]
for dt in datelist:

  df_tmp = df_new.loc[df_new.index <=dt].copy()
  df_orig = df_tmp.copy()

  # print progress
  if dt.day == 1:
    print(dt)

  # transform
  transform_stationary(df_tmp,features_to_transform,transform)

  #prepare
  idx_dates, array_y, array_X = prepare_data(df_tmp,n_steps,features)

  # scale the input and outputs
  scaler_X = MinMaxScaler(feature_range=(0,1))
  scaled_X = scaler_X.fit_transform(array_X)
  scaler_y = MinMaxScaler(feature_range=(0,1))
  scaled_y = scaler_y.fit_transform(array_y)


  # make predictions
  df_pred = make_ensemble_predictions(df_tmp,model,scaler_y,scaled_X,n_steps,len(features),n_predict,df_tmp.index.max(),weights)

  # get the previous df_hist data
  df_hist = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_ensemble_hist.pickle')

  # update the hist file with yesterdays close price, and add the new predictions
  df_hist_new =roll_ensemble_predictions(df_orig,df_pred,df_hist)
  df_hist_new.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_ensemble_hist.pickle')


# plot
plot_actual_ensemble(stock,df_hist_new)
df_hist_new.loc[start_date:end_date]

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


2021-02-01 00:00:00
2021-03-01 00:00:00
2021-04-01 00:00:00
2021-05-01 00:00:00
2021-06-01 00:00:00
2021-07-01 00:00:00
2021-08-01 00:00:00
2021-09-01 00:00:00


,actual,last_close,last_close_diff,last_close_cum,lstm,lstm_diff,lstm_cum,fbp,fbp_diff,fbp_cum,ens,ens_diff,ens_cum
2021-01-04,31988.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,33949.53,31988.71,-1960.82,-1960.82,27549.367188,-6400.162812,-6400.162812,22803.546669,-11145.983331,-11145.983331,25895.613657,-8053.916343,-8053.916343
2021-01-06,36769.36,33949.53,-2819.83,-4780.65,28562.408203,-8206.951797,-14607.114609,23255.601776,-13513.758224,-24659.741555,26757.564905,-10011.795095,-18065.711438
2021-01-07,39432.28,36769.36,-2662.92,-7443.57,30610.335938,-8821.944062,-23429.058672,23537.493506,-15894.786494,-40554.528049,28097.728970,-11334.551030,-29400.262468
2021-01-08,40582.81,39432.28,-1150.53,-8594.10,32760.267578,-7822.542422,-31251.601094,24123.665321,-16459.144679,-57013.672729,29603.068381,-10979.741619,-40380.004087
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-24,42810.57,44865.26,2054.69,-10821.86,43961.378906,1150.808906,-236499.774141,47394.461210,4583.891210,220459.606139,45589.568893,2778.998893,-8722.183911
2021-09-25,42670.64,42810.57,139.93,-10681.93,43927.261719,1256.621719,-235243.152422,46412.602061,3741.962061,224201.568201,44920.106888,2249.466888,-6472.717023
2021-09-26,43160.90,42670.64,-490.26,-11172.19,43882.335938,721.435937,-234521.716484,45723.420034,2562.520034,226764.088235,44577.464050,1416.564050,-5056.152973
2021-09-27,42147.35,43160.90,1013.55,-10158.64,43880.410156,1733.060156,-232788.656328,45000.093273,2852.743273,229616.831507,44304.988992,2157.638992,-2898.513981


### VMWare

In [ ]:
stock = 'VMW'
features_to_transform = ['open','high','low','close','b-upper','b-lower','b-middle','SPY','SKYY','VGT'] 
transform = 'log'
n_steps = 25
n_predict = 3
config ={'Economic':
         {'TREASURY_YIELD':[
                            {'interval':'daily','maturity':'10year','name':'yield10y'},
                            {'interval':'daily','maturity':'30year','name':'yield30y'},
                            ],
          'NONFARM_PAYROLL':{'interval':'monthly','name':'nfp'}

          },
         'Technical':{
           'BBANDS':{'interval':'daily','time_period':10},
           'RSI':{'interval':'daily','time_period':7},
           },
         'Commodities':['SPY','SKYY','VGT']
         }


# get full data history
df_vmw = get_consolidated_stock_data(stock,key,config,'full')
df_new = df_vmw.copy()

# get the trained lstm model
model = keras.models.load_model(f'/content/drive/MyDrive/Colab Notebooks/models/model_{stock}_final')


# Get the features used in the final model
df_features = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_features.pickle')
features = [f for f in df_features.columns if f not in ['symbol']]


# initial hist file
start_date = '2021-01-04'
end_date = '2021-10-07'
weights = {'last_close':0.4974155,'fbp':0.002427,'lstm':0.500158}
df_tmp = df_new.loc[df_new.index <=start_date]
df_hist_new =roll_ensemble_predictions(df_tmp)
df_hist_new.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_ensemble_hist.pickle')


# backtest for sep 2021
datelist = df_new.loc[end_date:start_date].index
datelist = datelist[::-1]
for dt in datelist:

  df_tmp = df_new.loc[df_new.index <=dt].copy()
  df_orig = df_tmp.copy()

  # print progress
  if dt.day == 1:
    print(dt)

  # transform
  transform_stationary(df_tmp,features_to_transform,transform)

  #prepare
  idx_dates, array_y, array_X = prepare_data(df_tmp,n_steps,features)

  # scale the input and outputs
  scaler_X = MinMaxScaler(feature_range=(0,1))
  scaled_X = scaler_X.fit_transform(array_X)
  scaler_y = MinMaxScaler(feature_range=(0,1))
  scaled_y = scaler_y.fit_transform(array_y)


  # make predictions
  df_pred = make_ensemble_predictions(df_tmp,model,scaler_y,scaled_X,n_steps,len(features),n_predict,df_tmp.index.max(),weights)


  # get the previous df_hist data
  df_hist = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_ensemble_hist.pickle')

  # update the hist file with yesterdays close price, and add the new predictions
  df_hist_new =roll_ensemble_predictions(df_orig,df_pred,df_hist)
  df_hist_new =roll_ensemble_predictions(df_orig,df_pred,df_hist)
  df_hist_new.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_ensemble_hist.pickle')


# plot
plot_actual_ensemble(stock,df_hist_new)
df_hist_new.loc[start_date:end_date]


### BLX.TO

In [ ]:
# ['PBD', 'VXX', 'b-upper', 'b-lower', 'b-middle', 'rsi', 'macd_hist', 'macd', 'yield3m_shift', 'unemployment_shift']
stock = 'BLX.TO'
features_to_transform = ['open','high','low','close','b-upper','b-lower','b-middle','PBD','VXX'] 
transform = 'log'
n_steps = 25
n_predict = 3
config ={'Economic':
         {'TREASURY_YIELD':[{'interval':'daily','maturity':'3month','name':'yield3m'}],
          'UNEMPLOYMENT':{'interval':'monthly','name':'unemployment'}
          },
         'Technical':{
           'BBANDS':{'interval':'daily','time_period':10},
           'RSI':{'interval':'daily','time_period':7},
           'MACD':{'interval':'daily','time_period':None}
           },
         'Commodities':['VXX','PBD']
         }

# get new data
df_blx = get_consolidated_stock_data(stock,key,config,'full')
df_new = df_blx.copy()


# # shift features to match the model
df_new = shift_features(df_new,{'yield3m': -60,'unemployment':-60})

# get the trained lstm model
model = keras.models.load_model(f'/content/drive/MyDrive/Colab Notebooks/models/model_{stock}_final')


# Get the features used in the final model
df_features = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_features.pickle')
features = [f for f in df_features.columns if f not in ['symbol']]


# initial hist file
start_date = '2021-01-04'
end_date = '2021-10-07'
df_tmp = df_new.loc[df_new.index <=start_date]
df_hist_new =roll_ensemble_predictions(df_tmp)
df_hist_new.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_ensemble_hist.pickle')

# backtest for sep 2021
datelist = df_new.loc[end_date:start_date].index
datelist = datelist[::-1]
for dt in datelist:

  df_tmp = df_new.loc[df_new.index <=dt].copy()
  df_orig = df_tmp.copy()

  # print progress
  if dt.day == 1:
    print(dt)

  # transform
  transform_stationary(df_tmp,features_to_transform,transform)

  #prepare
  idx_dates, array_y, array_X = prepare_data(df_tmp,n_steps,features)

  # scale the input and outputs
  scaler_X = MinMaxScaler(feature_range=(0,1))
  scaled_X = scaler_X.fit_transform(array_X)
  scaler_y = MinMaxScaler(feature_range=(0,1))
  scaled_y = scaler_y.fit_transform(array_y)


  # make predictions
  df_pred = make_ensemble_predictions(df_tmp,model,scaler_y,scaled_X,n_steps,len(features),n_predict,df_tmp.index.max(),weights)

  # get the previous df_hist data
  df_hist = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_ensemble_hist.pickle')

  # update the hist file with yesterdays close price, and add the new predictions
  df_hist_new =roll_ensemble_predictions(df_orig,df_pred,df_hist)
  df_hist_new.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_ensemble_hist.pickle')


# plot
plot_actual_ensemble(stock,df_hist_new)
df_hist_new.loc[start_date:end_date]